In [1]:
import re, os, string, warnings, json
warnings.filterwarnings("ignore")
import math
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt', quiet=True)
from collections import Counter
from rapidfuzz import fuzz
from word2number import w2n
import dateparser
from dateutil import parser as duparser

In [2]:
df = pd.read_csv("synthetic_dataset.csv", encoding_errors="replace")

In [4]:
# This is being done so as to confirm to a same schema for data throughout the project
# Heuristic rename if your headers differ slightly
rename_map = {
    "reference": "gold_reference",
    "gold": "gold_reference",
    "answer": "ai_response",
    "response": "ai_response",
}
for k,v in rename_map.items():
    if k in df.columns and v not in df.columns:
        df.rename(columns={k:v}, inplace=True)

# Ensure exactly the 4 we use exist
for c in ["question","context","gold_reference","ai_response"]:
    if c not in df.columns: df[c] = ""

def _clean(s): 
    return (str(s) if pd.notna(s) else "").strip()

UNANSWERABLE_MARKERS = {
    "unanswerable","not answerable","cannot be determined","insufficient context",
    "not available in passage","unknown","na","n/a"
}

def derive_flags(row):
    gold = _clean(row["gold_reference"]).lower()
    ctx  = _clean(row["context"])
    is_unanswerable = gold in UNANSWERABLE_MARKERS
    context_missing = (ctx == "")
    no_gold = (gold == "")  # empty gold means judge vs context only
    return pd.Series({
        "is_unanswerable": is_unanswerable,
        "context_missing": context_missing,
        "no_gold": no_gold
    })

flags = df.apply(derive_flags, axis=1)
df = pd.concat([df, flags], axis=1)
df.head(3)

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  is_unanswerable  \
0                    The passage doesn’t state this.            False   
1  It states that what did the court of justice w...            False   
2  The decision was influenced by the Paris Conve...            False   

   context_missing  no_gold  is_unanswerable  context_missing  no_gold  
0            False     True            False            False     True  
1            False     True            False            False     True  
2            False     True            False            False     True

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# ---------- custom functions for various use cases ----------
def to_text(x) -> str:
    if x is None:
        return ""
    try:
        import pandas as _pd
        if _pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x)

def normalize_text(s: str) -> str:
    s = to_text(s).strip().lower()
    return re.sub(r"\s+", " ", s)

def safe_bool(x) -> bool:
    s = to_text(x).strip().lower()
    if s in {"1","true","yes","y","t"}: return True
    return False

def sent_split(raw: str):
    raw = to_text(raw)
    parts = re.split(r"[\.?!;\n]+", raw)
    return [p.strip() for p in parts if p and p.strip()]

def extract_numbers(norm: str):
    return re.findall(r"\d+(?:\.\d+)?", to_text(norm))

def extract_entities_simple(raw: str):
    """Heuristic: multi-word Proper Nouns + single capitalized tokens from RAW string."""
    raw = to_text(raw)
    multi = re.findall(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b", raw)
    single = re.findall(r"\b[A-Z][a-z]+\b", raw)
    ents = set(single)
    for phrase in multi:
        ents.add(phrase)
        ents.update(phrase.split())
    return list(ents)

SPECULATIVE = {
    "probably","likely","appears","seems","may","might","could","assume",
    "assuming","suggests","it is possible","typically","usually","generally",
    "perhaps","apparently","reportedly","supposedly","presumably"
}

def has_speculative(resp_norm: str) -> int:
    r = resp_norm
    count = 0
    for w in SPECULATIVE:
        # whole word or phrase
        if re.search(rf"\b{re.escape(w)}\b", r):
            count += 1
    return count

def no_answer_template(resp_norm: str) -> bool:
    s = resp_norm
    pats = [
        "no answer", "not in passage", "not within passage", "not stated",
        "cannot be determined", "cannot be found", "insufficient information",
        "unknown", "not specified", "no information", "answer not available",
        "cannot determine", "not provided", "information is missing"
    ]
    return any(p in s for p in pats)


In [7]:
# Assumption scoring based on following parameters:
# 1) presence of entities or numbers ot present in context
# 2) content overlap ratio
# 3) using speculative language (words like "maybe", "seems to be","probably", "assuming")
# 4) Similarity scores between context and ai_reponse
# cases where question is unanswerable, high penalty is applied for giving answer other than "no answer" or such texts.
# designing is such that --> higher assumption score == less ssumption 

def subscores_assumption(context_raw: str, response_raw: str) -> dict:
    ctx_raw = to_text(context_raw)
    resp_raw = to_text(response_raw)
    ctx_norm = normalize_text(ctx_raw)
    resp_norm = normalize_text(resp_raw)

    # (1) New Claim Detection (entities & numbers novel to context)
    resp_ents = set(extract_entities_simple(resp_raw))
    ctx_ents  = set(extract_entities_simple(ctx_raw))
    resp_nums = set(extract_numbers(resp_norm))
    ctx_nums  = set(extract_numbers(ctx_norm))

    total_claims = max(len(resp_ents) + len(resp_nums), 1)
    new_claims = sum(1 for e in resp_ents if e and e not in ctx_ents) \
               + sum(1 for n in resp_nums if n not in ctx_nums)
    new_claim_ratio = new_claims / total_claims      # higher = worse
    ucr = 1.0 - new_claim_ratio                      # Unsupported Claim Ratio score (higher = better)

    # (2) Content Overlap Ratio
    resp_tokens = set(resp_norm.split())
    ctx_tokens  = set(ctx_norm.split())
    content_overlap = len(resp_tokens & ctx_tokens) / (len(resp_tokens) or 1)

    # (3) Speculative Language Penalty
    spec_count = has_speculative(resp_norm)
    # map count to penalty in [0,1] → more spec = lower score
    # up to 3 occurrences reduce linearly to 0
    spec_score = max(0.0, 1.0 - min(spec_count, 3)/3.0)

    # (4) Sentence→Context TF-IDF MaxSim (avg of best per response sentence)
    ssms = 0.0
    r_sents = sent_split(resp_raw)
    c_sents = sent_split(ctx_raw)
    if r_sents and c_sents:
        # build TF-IDF on all sentences for stable vocab
        docs = c_sents + r_sents
        vect = TfidfVectorizer(min_df=1, ngram_range=(1,2))
        try:
            X = vect.fit_transform(docs)
            C = X[:len(c_sents)]
            R = X[len(c_sents):]
            sims = cosine_similarity(R, C)  # [R,S]
            best = sims.max(axis=1) if sims.size else np.array([0.0])
            ssms = float(np.clip(best.mean(), 0.0, 1.0))
        except ValueError:
            ssms = 0.0

    return {
        "UCR": float(ucr),
        "CONTENT_OVERLAP": float(content_overlap),
        "SPECULATIVE_SCORE": float(spec_score),
        "SSMS": float(ssms),
        "new_claims": int(new_claims),
        "total_claims": int(total_claims)
    }

def assumption_control_score(row: pd.Series,
                             weights=(0.35, 0.20, 0.15, 0.30),  # (UCR, Overlap, Speculative, SSMS)
                             tau_label=0.75):
    """
    Assumption Control uses only CONTEXT (no gold).
    If context missing → neutral 0.5.
    """
    ctx  = to_text(row.get("context",""))
    resp = to_text(row.get("ai_response",""))
    context_missing = safe_bool(row.get("context_missing", False))
    is_unans        = safe_bool(row.get("is_unanswerable", False))

    if context_missing or not ctx.strip():
        return {
            "assump_score": 0.5,
            "assump_label": int(0.5 >= tau_label),
            "UCR": np.nan, "CONTENT_OVERLAP": np.nan, "SPECULATIVE_SCORE": np.nan, "SSMS": np.nan,
            "new_claims": np.nan, "total_claims": np.nan,
            "no_answer_template": no_answer_template(normalize_text(resp)),
            "support_used": "none"
        }

    subs = subscores_assumption(ctx, resp)
    UCR, OVER, SPEC, SSMS = subs["UCR"], subs["CONTENT_OVERLAP"], subs["SPECULATIVE_SCORE"], subs["SSMS"]
    w_u, w_o, w_s, w_m = weights
    score = w_u*UCR + w_o*OVER + w_s*SPEC + w_m*SSMS

    # Unanswerable handling
    resp_norm = normalize_text(resp)
    template = no_answer_template(resp_norm)

    # If unanswerable and they *still* add new claims → cap
    if is_unans and subs["new_claims"] > 0:
        score = min(score, 0.5)
    # If unanswerable and they correctly refuse with no extra claims → small bonus
    if is_unans and template and subs["new_claims"] == 0:
        score = min(1.0, score + 0.1)

    score = float(min(1.0, max(0.0, score)))
    label = int(score >= tau_label)

    return {
        "assump_score": score,
        "assump_label": label,
        **subs,
        "no_answer_template": template,
        "support_used": "context"
    }

def add_assumption_scores(df: pd.DataFrame,
                          weights=(0.35, 0.20, 0.15, 0.30),
                          tau_label=0.75,
                          return_diagnostics=True) -> pd.DataFrame:
    rows = [assumption_control_score(r, weights=weights, tau_label=tau_label) for _, r in df.iterrows()]
    diag = pd.DataFrame(rows)
    if not return_diagnostics:
        diag = diag[["assump_score","assump_label"]]
    return pd.concat([df.reset_index(drop=True), diag], axis=1)


In [8]:
df = add_assumption_scores(df, weights=(0.35,0.20,0.15,0.30), tau_label=0.75, return_diagnostics=True)
df[["ai_response","assump_score","assump_label","UCR","CONTENT_OVERLAP","SPECULATIVE_SCORE","SSMS","no_answer_template"]].head()

ai_response  assump_score  \
0                    The passage doesn’t state this.      0.604477   
1  It states that what did the court of justice w...      0.281990   
2  The decision was influenced by the Paris Conve...      0.278720   
3                               No answer in passage      0.268096   
4  Bullet 1) The Court accepted horizontal effect...      0.384162   

   assump_label       UCR  CONTENT_OVERLAP  SPECULATIVE_SCORE      SSMS  \
0             0  1.000000         0.400000                1.0  0.081591   
1             0  0.000000         0.437500                1.0  0.148299   
2             0  0.200000         0.176471                1.0  0.078087   
3             0  0.000000         0.500000                1.0  0.060321   
4             0  0.428571         0.312500                1.0  0.072206   

   no_answer_template  
0               False  
1               False  
2               False  
3                True  
4               False

In [9]:
df.to_csv("scored datasets/assumption_scores_added.csv")